# NexusFlow Complete Demo: Multi-Table ML with Advanced Features

Welcome to NexusFlow! This notebook demonstrates how to use NexusFlow's advanced multi-table machine learning capabilities with synthetic data. You'll learn how to train sophisticated models that can learn from multiple related datasets simultaneously.

## What is NexusFlow?

NexusFlow is a cutting-edge machine learning framework designed for **multi-table relational learning**. Unlike traditional ML approaches that work with single flattened datasets, NexusFlow can:

- 🔗 **Learn from multiple related tables simultaneously**
- 🧠 **Use advanced transformer architectures** (FT-Transformer, TabNet, Standard)
- ⚡ **Leverage FlashAttention** for efficient processing
- 🔀 **Apply Mixture of Experts (MoE)** for complex patterns
- 📊 **Advanced preprocessing pipelines** with automatic feature detection
- 🎯 **Cross-contextual attention** to capture relationships between tables
- 📦 **Model optimization** with quantization and pruning

Let's dive in!

## Section 1: Environment Setup

First, let's set up our environment and import the necessary libraries.

In [ ]:
!git clone https://github.com/ArkChaudhary/NexusFlow.git
!pip install -r NexusFlow/requirements.txt
!pip install loguru

In [ ]:
# Core imports
import pandas as pd
import numpy as np
import os
import json
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# NexusFlow imports
import sys
sys.path.insert(0, './NexusFlow/src')
from nexusflow.project_manager import ProjectManager
from nexusflow.config import load_config_from_file
from nexusflow.trainer.trainer import Trainer
from nexusflow.api.model_api import load_model, extract_pytorch_model, create_optimized_artifact, ModelAPI
from nexusflow.optimization.optimizer import optimize_model

print("🚀 NexusFlow environment ready!")
print("✅ All imports successful")

## Section 2: Project Initialization & Data Preparation

### Setting up the Project Structure

NexusFlow follows a standardized project structure that keeps everything organized:

```
nexusflow_project/
├── configs/          # Configuration files
├── datasets/         # Raw data files
├── models/          # Saved model artifacts
├── results/         # Training results and logs
├── notebooks/       # Jupyter notebooks
└── src/            # Source code (if extending)
```

Let's create this structure and populate it with synthetic data for our demo.

In [ ]:
# Initialize project structure
pm = ProjectManager()
project_path = pm.init_project('nexusflow_colab_demo', force=True)
datasets_path = os.path.join(project_path, 'datasets')
os.makedirs(datasets_path, exist_ok=True)

In [ ]:
# Change to project directory
%cd nexusflow_colab_demo
print(f"📁 Project created at: {os.getcwd()}")
print("📂 Project structure:")
for root, dirs, files in os.walk('.'):
    level = root.replace('.', '').count(os.sep)
    indent = ' ' * 2 * level
    print(f'{indent}{os.path.basename(root)}/')

### Creating Synthetic Multi-Table Data

For this tutorial, we'll create two related datasets:
1. **`users.csv`** - User profile information (demographics, preferences)  
2. **`transactions.csv`** - User transaction history (purchases, amounts, categories)

Our goal will be to predict user **churn risk** based on both profile and transaction patterns.

In [ ]:
# Set random seed for reproducibility
np.random.seed(42)

# Generate Users Dataset
n_users = 1000
user_ids = range(1, n_users + 1)

users_data = {
    'user_id': user_ids,
    'age': np.random.normal(35, 12, n_users).astype(int).clip(18, 80),
    'income': np.random.lognormal(10.5, 0.5, n_users).astype(int).clip(20000, 200000),
    'account_tenure_months': np.random.exponential(24, n_users).astype(int).clip(1, 120),
    'num_products': np.random.poisson(2.5, n_users).clip(1, 8),
    'region': np.random.choice(['North', 'South', 'East', 'West', 'Central'], n_users, 
                              p=[0.2, 0.25, 0.2, 0.15, 0.2]),
    'subscription_tier': np.random.choice(['Basic', 'Premium', 'Enterprise'], n_users, 
                                        p=[0.5, 0.35, 0.15]),
    'support_tickets': np.random.poisson(1.2, n_users),
}

users_df = pd.DataFrame(users_data)

In [ ]:
# Generate Transactions Dataset
n_transactions = 5000
transaction_user_ids = np.random.choice(user_ids, n_transactions, 
                                      p=np.random.dirichlet(np.ones(n_users)))

transactions_data = {
    'user_id': transaction_user_ids,
    'transaction_amount': np.random.lognormal(4, 1, n_transactions).clip(10, 5000),
    'transaction_frequency': np.random.exponential(2, n_transactions).clip(0.1, 20),
    'days_since_last_transaction': np.random.exponential(7, n_transactions).astype(int).clip(0, 365),
    'payment_method': np.random.choice(['Credit', 'Debit', 'Digital', 'Cash'], n_transactions,
                                     p=[0.4, 0.3, 0.25, 0.05]),
    'transaction_category': np.random.choice(['Retail', 'Services', 'Entertainment', 'Food', 'Other'], 
                                           n_transactions, p=[0.3, 0.25, 0.2, 0.15, 0.1]),
    'merchant_rating': np.random.beta(8, 2, n_transactions) * 5,  # Skewed toward high ratings
}

transactions_df = pd.DataFrame(transactions_data)

In [ ]:
# Create synthetic churn labels with realistic patterns
# Higher churn probability for: low income, few products, high support tickets, low transaction frequency
churn_probability = (
    0.1 +  # Base rate
    0.15 * (users_df['income'] < 40000) +  # Low income
    0.1 * (users_df['num_products'] == 1) +  # Single product
    0.05 * (users_df['support_tickets'] > 2) +  # Many support tickets
    0.1 * (users_df['account_tenure_months'] < 6)  # New accounts
)

In [ ]:
# Add some transaction-based features for churn
user_tx_summary = transactions_df.groupby('user_id').agg({
    'transaction_frequency': 'mean',
    'days_since_last_transaction': 'min'
}).reset_index()

users_df = users_df.merge(user_tx_summary, on='user_id', how='left')
users_df['transaction_frequency'] = users_df['transaction_frequency'].fillna(0)
users_df['days_since_last_transaction'] = users_df['days_since_last_transaction'].fillna(365)

In [ ]:
# Adjust churn probability based on transaction patterns
churn_probability += 0.2 * (users_df['transaction_frequency'] < 0.5)  # Low frequency
churn_probability += 0.15 * (users_df['days_since_last_transaction'] > 90)  # Long absence

users_df['churn_risk'] = np.random.binomial(1, churn_probability.clip(0, 0.8), n_users)

In [ ]:
# Drop helper columns used for churn generation
users_df = users_df.drop(['transaction_frequency', 'days_since_last_transaction'], axis=1)

In [ ]:
# Save datasets
users_df.to_csv('datasets/users.csv', index=False)
transactions_df.to_csv('datasets/transactions.csv', index=False)

print("📊 Synthetic datasets created successfully!")
print(f"👥 Users dataset: {len(users_df):,} records, {len(users_df.columns)} columns")
print(f"💳 Transactions dataset: {len(transactions_df):,} records, {len(transactions_df.columns)} columns")
print(f"🎯 Churn rate: {users_df['churn_risk'].mean():.1%}")

# Preview the data
print("\n📋 Users dataset preview:")
print(users_df.head())
print("\n📋 Transactions dataset preview:")  
print(transactions_df.head())

## Section 3: Configuration

### Configuring the Model

NexusFlow is configured using a single YAML file that specifies:
- **Project settings** (name, primary key, target variable)
- **Dataset configurations** (transformer types, complexity levels)
- **Architecture settings** (embedding dimensions, refinement iterations)
- **Advanced features** (MoE, FlashAttention, preprocessing)
- **Training parameters** (batch size, epochs, optimization)

Let's create a sophisticated configuration that showcases NexusFlow's advanced capabilities:

In [ ]:
%%writefile configs/config.yaml
project_name: "nexusflow_churn_prediction"
primary_key: "user_id"

target:
  target_column: "churn_risk"
  target_table: "users.csv"

# Dataset configurations with different transformer types
datasets:
  - name: "users.csv"
    transformer_type: "ft_transformer"  # Feature Tokenizer Transformer for tabular data
    complexity: "medium"
    context_weight: 1.0
    categorical_columns: ["region", "subscription_tier"]
    numerical_columns: ["age", "income", "account_tenure_months", "num_products", "support_tickets"]
  
  - name: "transactions.csv"
    transformer_type: "standard"  # Standard transformer with FlashAttention
    complexity: "medium"
    context_weight: 0.8
    categorical_columns: ["payment_method", "transaction_category"]
    numerical_columns: ["transaction_amount", "transaction_frequency", "days_since_last_transaction", "merchant_rating"]

# Architecture configuration
architecture:
  global_embed_dim: 128
  refinement_iterations: 4
  use_moe: true              # Enable Mixture of Experts
  num_experts: 6             # Number of expert networks
  use_flash_attn: true       # Enable FlashAttention optimization
  top_k_contexts: 3          # Limit cross-attention to top-3 most relevant contexts

# Training configuration
training:
  batch_size: 32
  epochs: 15
  
  optimizer:
    name: "adam"
    lr: 0.001
    weight_decay: 0.0001
  
  split_config:
    test_size: 0.2
    validation_size: 0.2
    randomize: true
  
  # Synthetic data options
  use_synthetic: false
  synthetic:
    n_samples: 256
    feature_dim: 5
  
  # Advanced preprocessing
  use_advanced_preprocessing: true
  auto_detect_types: true
  
  # Training optimizations
  early_stopping: true
  patience: 5
  gradient_clipping: 1.0

# MLOps configuration
mlops:
  logging_provider: "stdout"
  experiment_name: "churn_prediction_demo"
  log_attention_patterns: false      

In [ ]:
print("⚙️ Configuration file created successfully!")
print("📝 Key configuration highlights:")
print("   🔄 FT-Transformer for users table (advanced tabular processing)")  
print("   ⚡ Standard transformer with FlashAttention for transactions")
print("   🔀 Mixture of Experts enabled (6 experts)")
print("   🧠 Advanced preprocessing with auto-detection")
print("   🎯 Binary classification: churn risk prediction")
print("   📊 4 refinement iterations for cross-table learning")

## Section 4: Training the Model

### Let's Train!

With our data and configuration ready, training a sophisticated multi-table model is remarkably simple. NexusFlow handles all the complexity behind the scenes:

- **Automatic data loading and preprocessing**
- **Cross-table relationship modeling**
- **Advanced transformer architectures**
- **Mixture of Experts routing**
- **FlashAttention optimization**

In [ ]:
# Load configuration and initialize trainer
config = load_config_from_file('configs/config.yaml')
trainer = Trainer(config, work_dir='.')

print("🎯 Starting NexusFlow training with advanced features...")
#print(f"📊 Model architecture: {config.architecture.num_experts} MoE experts, {config.architecture.global_embed_dim}D embeddings")
print(f"⚡ Optimizations: FlashAttention={config.architecture.use_flash_attn}, MoE={config.architecture.use_moe}")
print(f"🔄 Cross-table refinement: {config.architecture.refinement_iterations} iterations")

# Run training
trainer.train()

print("\n🎉 Training completed successfully!")
print("📁 Artifacts created:")
print("   📦 nexusflow_churn_prediction.nxf (complete model artifact)")
print("   💾 best_model.pt (PyTorch checkpoint)")
print("   📊 results/training_history.json (training metrics)")
print("   🔧 preprocessing/ (preprocessing artifacts)")

## Section 5: Using the Trained Model

### Model Loading and Inference

Our training produced a complete `.nxf` model artifact that contains:
- **The trained NexusFormer model**
- **All preprocessing transformations**
- **Model configuration and metadata**
- **Performance metrics**

Let's load this artifact and make predictions on new data:

In [ ]:
# Load the trained model artifact
model = load_model('nexusflow_churn_prediction.nxf')

print("📦 Model artifact loaded successfully!")
print("\n📋 Model Summary:")
print(model.summary())

# Get detailed model parameters
params = model.get_params()
print(f"\n🔍 Model Details:")
print(f"   Total Parameters: {params['total_parameters']:,}")
print(f"   Architecture: {params['model_class']} with {params['architecture']['num_encoders']} encoders")
print(f"   Training Accuracy: {params['training_info']['best_val_metric']:.4f}")
print(f"   Preprocessing: {'Enabled' if params.get('phase_2_features', {}).get('advanced_preprocessing', False) else 'Basic'}")

In [ ]:
# Create new sample data for inference (simulating new users and their transactions)
np.random.seed(123)  # Different seed for new data

# New users data
new_users = pd.DataFrame({
    'user_id': [1001, 1002, 1003, 1004, 1005],
    'age': [28, 45, 36, 52, 31],
    'income': [45000, 85000, 62000, 120000, 38000],
    'account_tenure_months': [3, 24, 12, 48, 6],
    'num_products': [1, 3, 2, 5, 1],
    'region': ['North', 'South', 'East', 'West', 'Central'],
    'subscription_tier': ['Basic', 'Premium', 'Basic', 'Enterprise', 'Basic'],
    'support_tickets': [2, 0, 1, 0, 3]
})

In [ ]:
# New transactions data
new_transactions = pd.DataFrame({
    'user_id': [1001, 1002, 1003, 1004, 1005],
    'transaction_amount': [45.0, 89.99, 234.50, 1250.00, 15.99],
    'transaction_frequency': [0.5, 1.8, 1.2, 3.5, 0.2],
    'days_since_last_transaction': [45, 7, 14, 1, 89],
    'payment_method': ['Credit', 'Digital', 'Credit', 'Credit', 'Cash'],
    'transaction_category': ['Retail', 'Entertainment', 'Retail', 'Services', 'Other'],
    'merchant_rating': [4.2, 3.8, 4.5, 4.9, 3.2]
})

print("👤 New sample data created:")
print(new_users[['user_id', 'age', 'income', 'subscription_tier', 'support_tickets']])
print(f"\n💳 Associated transactions: {len(new_transactions)} records")

In [ ]:
# Make predictions using the multi-table format
prediction_data = {
    'users.csv': new_users,
    'transactions.csv': new_transactions  
}

# Get predictions (churn probabilities)
predictions = model.predict(prediction_data)

print("🎯 Churn Risk Predictions:")
print("=" * 50)

results_df = new_users[['user_id', 'age', 'income', 'subscription_tier']].copy()
results_df['churn_probability'] = predictions
results_df['risk_level'] = results_df['churn_probability'].apply(
    lambda x: 'High' if x > 0.7 else ('Medium' if x > 0.4 else 'Low')
)

for _, row in results_df.iterrows():
    print(f"User {row['user_id']:4d}: {row['churn_probability']:.3f} ({row['risk_level']:6s}) - "
          f"Age {row['age']}, {row['subscription_tier']:10s}, ${row['income']:,}")

print(f"\n📊 Risk Distribution:")
print(f"   High Risk (>70%):   {sum(results_df['risk_level'] == 'High')} users")
print(f"   Medium Risk (40-70%): {sum(results_df['risk_level'] == 'Medium')} users") 
print(f"   Low Risk (<40%):    {sum(results_df['risk_level'] == 'Low')} users")

### Model Optimization

NexusFlow includes state-of-the-art model optimization techniques to reduce model size and improve inference speed without sacrificing accuracy. Let's demonstrate both quantization and pruning:

In [ ]:
# Load the original trained model
original_artifact = load_model('nexusflow_churn_prediction.nxf')

print("🔧 Model Optimization Demonstration")
print("=" * 50)

params_info = original_artifact.get_params()
original_params = params_info['total_parameters']
original_size_mb = original_params * 4 / (1024 * 1024)

print(f"📊 Original Model:")
print(f"   Parameters: {original_params:,}")
print(f"   Size: {original_size_mb:.2f} MB")

# Extract the actual PyTorch model for optimization
actual_model = extract_pytorch_model(original_artifact)

2025-08-31 19:50:09.198 | INFO     | nexusflow.api.model_api:load:562 - Loading model with: encoder_type=standard, use_moe=True, num_experts=6, use_flash_attn=True


2025-08-31 19:50:09.224 | INFO     | nexusflow.model.nexus_former:__init__:553 - Enhanced NexusFormer initialized: 2 standard encoders, 4 iterations, MoE=True, FlashAttn=True
2025-08-31 19:50:09.232 | INFO     | nexusflow.api.model_api:load:577 - NexusFlow model artifact loaded from: nexusflow_churn_prediction.nxf
2025-08-31 19:50:09.232 | INFO     | nexusflow.api.model_api:load:579 - Loaded with preprocessors for: ['users.csv', 'transactions.csv']
2025-08-31 19:50:09.233 | INFO     | nexusflow.api.model_api:__init__:45 - NexusFlow model artifact initialized with 2 input dimensions
2025-08-31 19:50:09.234 | INFO     | nexusflow.api.model_api:__init__:47 - Preprocessors available for: ['users.csv', 'transactions.csv']


🔧 Model Optimization Demonstration
📊 Original Model:
   Parameters: 2,126,873
   Size: 8.11 MB


In [ ]:
# Demonstrate quantization
print("\n⚡ Applying Dynamic Quantization...")
quantized_pytorch_model, quant_metadata = optimize_model(actual_model, method='quantization')

print(f"\n📈 Quantization Results:")
print(f"   Method: {quant_metadata['method']}")
print(f"   Size Reduction: {quant_metadata['size_reduction']:.1%}")
print(f"   Parameter Reduction: {quant_metadata['parameter_reduction']:.1%}")
print(f"   Original Size: {quant_metadata['original_size_mb']:.2f} MB")
print(f"   Optimized Size: {quant_metadata['optimized_size_mb']:.2f} MB")

2025-08-31 19:50:13.548 | INFO     | nexusflow.optimization.optimizer:optimize_model:166 - 🚀 Starting model optimization with method: quantization
2025-08-31 19:50:13.553 | INFO     | nexusflow.optimization.optimizer:quantize_model:35 - 🔧 Starting dynamic quantization...
2025-08-31 19:50:13.558 | INFO     | nexusflow.optimization.optimizer:quantize_model:42 -    Original model size: 8.11 MB
2025-08-31 19:50:13.651 | INFO     | nexusflow.optimization.optimizer:quantize_model:64 - ✅ Dynamic quantization complete:
2025-08-31 19:50:13.652 | INFO     | nexusflow.optimization.optimizer:quantize_model:65 -    Quantized model size: 0.27 MB
2025-08-31 19:50:13.653 | INFO     | nexusflow.optimization.optimizer:quantize_model:66 -    Size reduction: 96.7%
2025-08-31 19:50:13.654 | INFO     | nexusflow.optimization.optimizer:quantize_model:67 -    Quantization time: 0.09s
2025-08-31 19:50:13.655 | INFO     | nexusflow.optimization.optimizer:quantize_model:68 -    Target layers: 71 Linear layers
20


⚡ Applying Dynamic Quantization...

📈 Quantization Results:
   Method: dynamic_quantization
   Size Reduction: 96.7%
   Parameter Reduction: 96.7%
   Original Size: 8.11 MB
   Optimized Size: 0.27 MB


In [ ]:
# Demonstrate pruning
print("\n✂️  Applying Global Unstructured Pruning...")
pruned_pytorch_model, prune_metadata = optimize_model(actual_model, method='pruning', amount=0.3)

print(f"\n📈 Pruning Results:")
print(f"   Method: {prune_metadata['method']}")
print(f"   Pruning Amount: 30%")
print(f"   Actual Size Reduction: {prune_metadata['size_reduction']:.1%}")
print(f"   Parameter Reduction: {prune_metadata['parameter_reduction']:.1%}")

2025-08-31 19:50:14.467 | INFO     | nexusflow.optimization.optimizer:optimize_model:166 - 🚀 Starting model optimization with method: pruning
2025-08-31 19:50:14.473 | INFO     | nexusflow.optimization.optimizer:prune_model:90 - ✂️  Starting global unstructured pruning (amount=30.0%)...
2025-08-31 19:50:14.476 | INFO     | nexusflow.optimization.optimizer:prune_model:99 -    Original parameters: 2,126,873
2025-08-31 19:50:14.479 | INFO     | nexusflow.optimization.optimizer:prune_model:100 -    Original model size: 8.11 MB
2025-08-31 19:50:14.482 | INFO     | nexusflow.optimization.optimizer:prune_model:111 -    Found 72 Linear layers to prune



✂️  Applying Global Unstructured Pruning...


2025-08-31 19:50:14.718 | INFO     | nexusflow.optimization.optimizer:prune_model:138 - ✅ Global unstructured pruning complete:
2025-08-31 19:50:14.719 | INFO     | nexusflow.optimization.optimizer:prune_model:139 -    Pruned parameters: 2,126,873
2025-08-31 19:50:14.720 | INFO     | nexusflow.optimization.optimizer:prune_model:140 -    Actual sparsity: 0.0%
2025-08-31 19:50:14.721 | INFO     | nexusflow.optimization.optimizer:prune_model:141 -    Size reduction: 0.0%
2025-08-31 19:50:14.722 | INFO     | nexusflow.optimization.optimizer:prune_model:142 -    New model size: 8.11 MB
2025-08-31 19:50:14.724 | INFO     | nexusflow.optimization.optimizer:prune_model:143 -    Pruning time: 0.24s
2025-08-31 19:50:14.725 | INFO     | nexusflow.optimization.optimizer:prune_model:144 -    Pruned layers: 72
2025-08-31 19:50:14.727 | INFO     | nexusflow.optimization.optimizer:optimize_model:210 - 🎯 Model optimization summary:
2025-08-31 19:50:14.728 | INFO     | nexusflow.optimization.optimizer:o


📈 Pruning Results:
   Method: global_unstructured_pruning
   Pruning Amount: 30%
   Actual Size Reduction: 0.0%
   Parameter Reduction: 0.0%


In [ ]:
# Create optimized artifacts properly
print("\n💾 Saving Optimized Models...")

# Create quantized artifact
quantized_artifact = create_optimized_artifact(original_artifact, quantized_pytorch_model, quant_metadata)
ModelAPI(quantized_pytorch_model, quantized_artifact.meta).save('nexusflow_churn_prediction_quantized.nxf')

# Create pruned artifact  
pruned_artifact = create_optimized_artifact(original_artifact, pruned_pytorch_model, prune_metadata)
ModelAPI(pruned_pytorch_model, pruned_artifact.meta).save('nexusflow_churn_prediction_pruned.nxf')

print("✅ Optimized model artifacts saved:")
print("   📦 nexusflow_churn_prediction_quantized.nxf")
print("   📦 nexusflow_churn_prediction_pruned.nxf")

2025-08-31 19:50:17.325 | INFO     | nexusflow.api.model_api:__init__:45 - NexusFlow model artifact initialized with 2 input dimensions
2025-08-31 19:50:17.328 | INFO     | nexusflow.api.model_api:__init__:47 - Preprocessors available for: ['users.csv', 'transactions.csv']
2025-08-31 19:50:17.330 | INFO     | nexusflow.api.model_api:__init__:45 - NexusFlow model artifact initialized with 2 input dimensions
2025-08-31 19:50:17.331 | INFO     | nexusflow.api.model_api:__init__:47 - Preprocessors available for: ['users.csv', 'transactions.csv']
2025-08-31 19:50:17.333 | INFO     | nexusflow.api.model_api:save:507 - Saving optimized model with dynamic_quantization
2025-08-31 19:50:17.333 | INFO     | nexusflow.api.model_api:save:508 -   Size reduction: 96.7%
2025-08-31 19:50:17.335 | INFO     | nexusflow.api.model_api:save:509 -   Parameter reduction: 96.7%
2025-08-31 19:50:17.392 | INFO     | nexusflow.api.model_api:save:518 - NexusFlow model artifact saved to: nexusflow_churn_prediction_


💾 Saving Optimized Models...
✅ Optimized model artifacts saved:
   📦 nexusflow_churn_prediction_quantized.nxf
   📦 nexusflow_churn_prediction_pruned.nxf


In [ ]:
# Compare all model sizes
import os
original_size = os.path.getsize('nexusflow_churn_prediction.nxf') / (1024 * 1024)
quantized_size = os.path.getsize('nexusflow_churn_prediction_quantized.nxf') / (1024 * 1024)  
pruned_size = os.path.getsize('nexusflow_churn_prediction_pruned.nxf') / (1024 * 1024)

print(f"\n📊 Final Model Comparison:")
print(f"   Original:  {original_size:.2f} MB (100%)")
print(f"   Quantized: {quantized_size:.2f} MB ({quantized_size/original_size*100:.1f}%)")
print(f"   Pruned:    {pruned_size:.2f} MB ({pruned_size/original_size*100:.1f}%)")


📊 Final Model Comparison:
   Original:  8.19 MB (100%)
   Quantized: 2.39 MB (29.2%)
   Pruned:    8.19 MB (100.0%)


## Section 6: Advanced Features Deep Dive

### Cross-Contextual Attention Analysis

One of NexusFormer's key innovations is its cross-contextual attention mechanism, which learns relationships between different tables. Let's examine how this works:

In [ ]:
# Visualize model architecture and attention patterns
print("🧠 NexusFormer Architecture Analysis")
print("=" * 50)

model = original_artifact

architecture_info = model.visualize_flow()

print(f"🏗️  Model Architecture:")
print(f"   Architecture Type: {architecture_info['architecture']}")
print(f"   Number of Contextual Encoders: {architecture_info['num_encoders']}")
print(f"   Input Dimensions: {architecture_info['input_dims']}")
print(f"   Refinement Iterations: {architecture_info['refinement_iterations']}")

print(f"\n🔄 Learning Process:")
print(f"   1. Each table (users, transactions) is processed by specialized encoders")
print(f"   2. Cross-contextual attention learns relationships between tables")
print(f"   3. {architecture_info['refinement_iterations']} refinement iterations enhance representations")
print(f"   4. Final fusion layer combines all contextual information")
print(f"   5. Prediction head outputs churn probability")

2025-08-31 19:54:34.784 | WARNING  | nexusflow.api.model_api:visualize_flow:421 - Interactive visualization not yet implemented
2025-08-31 19:54:34.785 | INFO     | nexusflow.api.model_api:visualize_flow:422 - Model architecture summary:
2025-08-31 19:54:34.785 | INFO     | nexusflow.api.model_api:visualize_flow:423 -   - 2 contextual encoders
2025-08-31 19:54:34.786 | INFO     | nexusflow.api.model_api:visualize_flow:424 -   - Input dimensions: [7, 6]
2025-08-31 19:54:34.787 | INFO     | nexusflow.api.model_api:visualize_flow:425 -   - Refinement iterations: 4
2025-08-31 19:54:34.789 | INFO     | nexusflow.api.model_api:visualize_flow:426 -   - Total parameters: 2126873
2025-08-31 19:54:34.790 | INFO     | nexusflow.api.model_api:visualize_flow:427 -   - Preprocessing: Available


🧠 NexusFormer Architecture Analysis
🏗️  Model Architecture:
   Architecture Type: NexusFormer
   Number of Contextual Encoders: 2
   Input Dimensions: [7, 6]
   Refinement Iterations: 4

🔄 Learning Process:
   1. Each table (users, transactions) is processed by specialized encoders
   2. Cross-contextual attention learns relationships between tables
   3. 4 refinement iterations enhance representations
   4. Final fusion layer combines all contextual information
   5. Prediction head outputs churn probability


### Preprocessing Pipeline Insights

In [ ]:
# Examine the preprocessing pipeline that was automatically applied
if hasattr(model, 'get_params'):
    params = model.get_params()
    if 'phase_2_features' in params and params['phase_2_features']['advanced_preprocessing']:
        print("\n🔧 Advanced Preprocessing Pipeline Applied:")
        print("=" * 50)
        
        preprocessing_datasets = params['phase_2_features']['preprocessor_datasets']
        print(f"✅ Datasets processed: {preprocessing_datasets}")
        print("🔍 Automatic feature type detection enabled")
        print("📊 Advanced categorical encoding applied")
        print("📈 Numerical feature normalization applied")
        print("🚫 Missing value imputation handled")
        
        print(f"\n📋 Feature Processing Summary:")
        for dataset in preprocessing_datasets:
            print(f"   {dataset.title()} table: Categorical & numerical features automatically detected")
    else:
        print("\n💡 Basic preprocessing was used (fillna-based)")


💡 Basic preprocessing was used (fillna-based)


## Section 7: Model Evaluation & Insights

### Comprehensive Model Analysis

In [ ]:
# Load training history and analyze model performance
with open('results/training_history.json', 'r') as f:
    training_data = json.load(f)

history = training_data['training_history']
best_epoch = training_data['best_epoch']
best_metric = training_data['best_metric']

print("📈 Training Performance Analysis")
print("=" * 50)
print(f"🏆 Best Model: Epoch {best_epoch} with validation loss {best_metric:.6f}")
print(f"📊 Total Epochs: {len(history)}")
print(f"⏰ Early Stopping: {'Yes' if training_data.get('early_stopped', False) else 'No'}")
print(f"🔧 Advanced Preprocessing: {'Yes' if training_data.get('preprocessing_enabled', False) else 'No'}")

📈 Training Performance Analysis
🏆 Best Model: Epoch 14 with validation loss 0.073231
📊 Total Epochs: 15
⏰ Early Stopping: No
🔧 Advanced Preprocessing: Yes


In [ ]:
# Extract metrics for visualization
epochs = [h['epoch'] for h in history]
train_losses = [h['train_loss'] for h in history]
val_losses = [h.get('val_loss', None) for h in history if h.get('val_loss') is not None]

print(f"\n📉 Training Progress:")
print(f"   Initial Training Loss: {train_losses[0]:.6f}")
print(f"   Final Training Loss: {train_losses[-1]:.6f}")
if val_losses:
    print(f"   Initial Validation Loss: {val_losses[0]:.6f}")
    print(f"   Best Validation Loss: {min(val_losses):.6f}")


📉 Training Progress:
   Initial Training Loss: 0.099689
   Final Training Loss: 0.075091
   Initial Validation Loss: 0.099898
   Best Validation Loss: 0.073231


In [ ]:
# Model evaluation metrics
eval_results = model.evaluate()
if eval_results:
    print(f"\n🎯 Model Performance Metrics:")
    for metric, value in eval_results.items():
        if isinstance(value, float):
            print(f"   {metric.replace('_', ' ').title()}: {value:.4f}")
        else:
            print(f"   {metric.replace('_', ' ').title()}: {value}")


🎯 Model Performance Metrics:
   Best Validation Metric: 0.0732
   Best Epoch: 14


## Section 8: Conclusion & Next Steps

### 🎉 Congratulations!

You've successfully completed the NexusFlow tutorial and experienced the power of advanced multi-table machine learning! Here's what you accomplished:

**✅ What You Built:**
- **Multi-table ML model** using users and transactions data
- **Advanced transformer architectures** (FT-Transformer + Standard)
- **Mixture of Experts** for handling complex patterns
- **FlashAttention optimization** for efficient processing
- **Cross-contextual learning** between related tables
- **Advanced preprocessing** with automatic feature detection
- **Model optimization** with quantization and pruning

**🚀 Key Takeaways:**
1. **Multi-table learning** can capture complex relationships that single-table models miss
2. **Advanced architectures** like MoE and FlashAttention provide both performance and efficiency
3. **Automatic preprocessing** reduces manual feature engineering overhead
4. **Model optimization** can dramatically reduce deployment costs
5. **NexusFlow's unified API** makes complex ML accessible

### Next Steps

```python
print("🎯 Next Steps & Advanced Usage:")
print("=" * 50)
print("1. 📊 Try with your own multi-table datasets")
print("2. 🔧 Experiment with different transformer types (TabNet, etc.)")  
print("3. 🎛️  Tune hyperparameters for your specific domain")
print("4. 🔀 Explore more MoE configurations")
print("5. 📈 Scale to larger datasets and more tables")
print("6. 🌐 Deploy optimized models to production")
print("7. 📚 Check the NexusFlow documentation for advanced features")

print(f"\n💡 Pro Tips:")
print("   • Use 'ft_transformer' for tables with mixed categorical/numerical features")
print("   • Enable MoE for datasets with complex, diverse patterns")
print("   • Apply model optimization before production deployment")
print("   • Experiment with refinement_iterations for better cross-table learning")
print("   • Monitor attention patterns for model interpretability")

print(f"\n📁 Your artifacts are ready for production:")
print("   📦 nexusflow_churn_prediction.nxf (original model)")
print("   ⚡ nexusflow_churn_prediction_quantized.nxf (optimized for speed)")
print("   🗜️  nexusflow_churn_prediction_pruned.nxf (optimized for size)")
```

### Resources

- **Documentation:** Check the NexusFlow docs for comprehensive guides
- **GitHub:** Explore the source code and contribute  
- **Community:** Join discussions and share your experiences
- **Examples:** Browse more advanced use cases and tutorials

**Happy modeling with NexusFlow!** 🚀✨